##**1.Install dependencies**

In [ ]:
# Install packages we need (duckdb, pandas, openpyxl, plotly)
# These do NOT pull conflicting protobuf versions.
!pip install --quiet duckdb pandas openpyxl plotly


##**2.Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
GDRIVE_ROOT = "/content/drive/MyDrive/colab_ticket_pipeline"
import os
os.makedirs(GDRIVE_ROOT, exist_ok=True)
print("Using Drive path:", GDRIVE_ROOT)
# If you don't want Drive, set GDRIVE_ROOT = "/content"


Mounted at /content/drive
Using Drive path: /content/drive/MyDrive/colab_ticket_pipeline


##**3.Upload Excel file**

In [ ]:
from google.colab import files
uploaded = files.upload()  # pick Sample Data file for Analysis_Jan'25.xlsx
filename = next(iter(uploaded.keys()))
print("Uploaded:", filename)


Saving Sample Data file for Analysis_Jan'25.xlsx to Sample Data file for Analysis_Jan'25.xlsx
Uploaded: Sample Data file for Analysis_Jan'25.xlsx


##**4.Inspect Excel sheets and convert to CSV**

In [12]:
import pandas as pd, shutil, os

src = "/content/" + filename
dst = os.path.join(ROOT, filename)
shutil.copy(src, dst)

# Inspect sheet names
xls = pd.ExcelFile(dst)
print("Available sheets:", xls.sheet_names)

# 👇 Pick the correct sheet here
sheet_name = xls.sheet_names[0]   # change if needed, e.g. "Data"
print("Using sheet:", sheet_name)

# Load sheet into DataFrame
df = pd.read_excel(dst, sheet_name=sheet_name)

# Preview first rows
print("Shape:", df.shape)
display(df.head(5))

# Save to CSV
csv_path = os.path.join(ROOT, "tickets.csv")
df.to_csv(csv_path, index=False)
print("CSV saved:", csv_path)


Available sheets: ['Raw Data']
Using sheet: Raw Data
Shape: (4999, 15)


,inc_business_service,inc_category,inc_number,inc_priority,inc_sla_due,inc_sys_created_on,inc_resolved_at,inc_assigned_to,inc_state,inc_cmdb_ci,inc_caller_id,inc_short_description,inc_assignment_group,inc_close_code,inc_close_notes
0,GRP56.34 PRXX (DigitalC Workplace management),Business Service,I68793614,3 - Moderate,UNKNOWN,2018-10-03 00:00:36,2018-10-03 10:00:39,sn int-test-svc Functional,Closed,GRP56.34 PRXX (DigitalC Workplace management),Event Management,Monitor status is error for ukwwnppcm0981_[PCM...,EngineeringSystems-Lev2,3rd Party Issue,Covered in Problem management.
1,GRP56.34 PRXX (DigitalC Workplace management),Business Service,I68793614,3 - Moderate,UNKNOWN,2018-10-03 00:00:36,2018-10-03 10:00:39,Emily White,Closed,GRP56.34 PRXX (DigitalC Workplace management),Event Management,Monitor status is error for ukwwnppcm0981_[PCM...,EngineeringSystems-Lev2,3rd Party Issue,Covered in Problem management.
2,NxGen Tech. Business SErvices - Backup,Hardware-Infrastructure,I68793615,3 - Moderate,UNKNOWN,2018-10-03 00:01:38,2018-10-03 00:16:32,prd-sql Functional,Closed,NaN,Event Management,IND01PXXRBK01: NBR: OracleDb:Backup : Status: ...,Bakup_Tek-SVC-Lev2,Other not on list,Resolving from CliqApps
3,NxGen Tech. Business SErvices - Backup,Hardware-Infrastructure,I68793615,3 - Moderate,UNKNOWN,2018-10-03 00:01:38,2018-10-03 00:16:32,Grace Harris,Closed,NaN,Event Management,IND01PXXRBK01: NBR: OracleDb:Backup : Status: ...,Bakup_Tek-SVC-Lev2,Other not on list,Resolving from CliqApps
4,NxGen Tech. Business SErvices - Backup,Hardware-Infrastructure,I68793616,3 - Moderate,UNKNOWN,2018-10-03 00:21:33,2018-10-03 01:36:27,Jack Mitchell,Closed,NaN,Event Management,IND01PXXRBK01: SLQ: OracleDb:Backup : Status: ...,Bakup_Tek-SVC-Lev2,Other not on list,Resolving from CliqApps


CSV saved: /content/drive/MyDrive/colab_ticket_pipeline/tickets.csv


##**5.Load CSV into DuckDB as raw_tickets**

In [13]:
import duckdb

DB_PATH = os.path.join(ROOT, "tickets.duckdb")
con = duckdb.connect(DB_PATH)

# Load CSV into table
con.execute(f"CREATE OR REPLACE TABLE raw_tickets AS SELECT * FROM read_csv_auto('{csv_path}');")

print("Tables:", con.execute("SHOW TABLES").df())
print("Row count:", con.execute("SELECT count(*) FROM raw_tickets").fetchone()[0])

# Peek data
display(con.execute("SELECT * FROM raw_tickets LIMIT 5").df())


Tables:           name
0  raw_tickets
Row count: 4999


,inc_business_service,inc_category,inc_number,inc_priority,inc_sla_due,inc_sys_created_on,inc_resolved_at,inc_assigned_to,inc_state,inc_cmdb_ci,inc_caller_id,inc_short_description,inc_assignment_group,inc_close_code,inc_close_notes
0,GRP56.34 PRXX (DigitalC Workplace management),Business Service,I68793614,3 - Moderate,UNKNOWN,2018-10-03 00:00:36,2018-10-03 10:00:39,sn int-test-svc Functional,Closed,GRP56.34 PRXX (DigitalC Workplace management),Event Management,Monitor status is error for ukwwnppcm0981_[PCM...,EngineeringSystems-Lev2,3rd Party Issue,Covered in Problem management.
1,GRP56.34 PRXX (DigitalC Workplace management),Business Service,I68793614,3 - Moderate,UNKNOWN,2018-10-03 00:00:36,2018-10-03 10:00:39,Emily White,Closed,GRP56.34 PRXX (DigitalC Workplace management),Event Management,Monitor status is error for ukwwnppcm0981_[PCM...,EngineeringSystems-Lev2,3rd Party Issue,Covered in Problem management.
2,NxGen Tech. Business SErvices - Backup,Hardware-Infrastructure,I68793615,3 - Moderate,UNKNOWN,2018-10-03 00:01:38,2018-10-03 00:16:32,prd-sql Functional,Closed,None,Event Management,IND01PXXRBK01: NBR: OracleDb:Backup : Status: ...,Bakup_Tek-SVC-Lev2,Other not on list,Resolving from CliqApps
3,NxGen Tech. Business SErvices - Backup,Hardware-Infrastructure,I68793615,3 - Moderate,UNKNOWN,2018-10-03 00:01:38,2018-10-03 00:16:32,Grace Harris,Closed,None,Event Management,IND01PXXRBK01: NBR: OracleDb:Backup : Status: ...,Bakup_Tek-SVC-Lev2,Other not on list,Resolving from CliqApps
4,NxGen Tech. Business SErvices - Backup,Hardware-Infrastructure,I68793616,3 - Moderate,UNKNOWN,2018-10-03 00:21:33,2018-10-03 01:36:27,Jack Mitchell,Closed,None,Event Management,IND01PXXRBK01: SLQ: OracleDb:Backup : Status: ...,Bakup_Tek-SVC-Lev2,Other not on list,Resolving from CliqApps


##**6.Create stg_tickets**

In [14]:
import re

# Get column list
cols = list(con.execute("SELECT * FROM raw_tickets LIMIT 1").df().columns)
print("Raw columns:", cols)

# Mapping rules
candidates = {
    "ticket_id":      ["ticket_id","ticket","number","incid","id"],
    "created_date":   ["created","sys_created","opened","created_on"],
    "resolved_date":  ["resolved","closed","resolved_on","closed_on"],
    "status":         ["status","state"],
    "assigned_group": ["assigned_group","assignment_group","group"],
    "technician":     ["assigned_to","technician","assignee"],
    "resolution_time_hrs": ["resolution_time","time_to_resolve","time_spent"],
    "category":       ["category"],
    "sub_category":   ["sub_category","subcategory"],
    "priority":       ["priority","urgency","severity"],
    "customer_impact": ["customer_impact","impact"]
}

def find_col(patterns, columns):
    for c in columns:
        low = c.lower()
        for p in patterns:
            if p in low:
                return c
    return None

mapping = {k: find_col(v, cols) for k,v in candidates.items()}
print("Column mapping:", mapping)

def col_or_null(actual_col):
    return f'"{actual_col}"' if actual_col else "NULL"

create_stg_sql = f"""
CREATE OR REPLACE TABLE stg_tickets AS
WITH raw AS (
  SELECT * FROM raw_tickets
),
deduped AS (
  SELECT *,
    row_number() OVER (PARTITION BY {col_or_null(mapping['ticket_id'])}
                       ORDER BY try_cast({col_or_null(mapping['resolved_date'])} AS TIMESTAMP) DESC NULLS LAST,
                                try_cast({col_or_null(mapping['created_date'])} AS TIMESTAMP) DESC NULLS LAST) AS rn
  FROM raw
),
parsed AS (
  SELECT
    {col_or_null(mapping['ticket_id'])} AS ticket_id,
    {col_or_null(mapping['category'])} AS category,
    {col_or_null(mapping['sub_category'])} AS sub_category,
    {col_or_null(mapping['priority'])} AS priority,
    try_cast({col_or_null(mapping['created_date'])} AS TIMESTAMP) AS created_ts,
    try_cast({col_or_null(mapping['resolved_date'])} AS TIMESTAMP) AS resolved_ts,
    {col_or_null(mapping['status'])} AS status,
    {col_or_null(mapping['assigned_group'])} AS assigned_group,
    {col_or_null(mapping['technician'])} AS technician,
    {col_or_null(mapping['resolution_time_hrs'])} AS resolution_time_hrs,
    {col_or_null(mapping['customer_impact'])} AS customer_impact
  FROM deduped
  WHERE rn = 1
)
SELECT *,
       extract(year from created_ts) AS created_year,
       extract(month from created_ts) AS created_month,
       CASE WHEN resolved_ts IS NOT NULL THEN 1 ELSE 0 END AS is_closed
FROM parsed;
"""

con.execute(create_stg_sql)
print("stg_tickets created:", con.execute("SELECT count(*) FROM stg_tickets").fetchone()[0])
display(con.execute("SELECT * FROM stg_tickets LIMIT 5").df())


Raw columns: ['inc_business_service', 'inc_category', 'inc_number', 'inc_priority', 'inc_sla_due', 'inc_sys_created_on', 'inc_resolved_at', 'inc_assigned_to', 'inc_state', 'inc_cmdb_ci', 'inc_caller_id', 'inc_short_description', 'inc_assignment_group', 'inc_close_code', 'inc_close_notes']
Column mapping: {'ticket_id': 'inc_number', 'created_date': 'inc_sys_created_on', 'resolved_date': 'inc_resolved_at', 'status': 'inc_state', 'assigned_group': 'inc_assignment_group', 'technician': 'inc_assigned_to', 'resolution_time_hrs': None, 'category': 'inc_category', 'sub_category': None, 'priority': 'inc_priority', 'customer_impact': None}
stg_tickets created: 1870


,ticket_id,category,sub_category,priority,created_ts,resolved_ts,status,assigned_group,technician,resolution_time_hrs,customer_impact,created_year,created_month,is_closed
0,I68793616,Hardware-Infrastructure,<NA>,3 - Moderate,2018-10-03 00:21:33,2018-10-03 01:36:27,Closed,Bakup_Tek-SVC-Lev2,Jack Mitchell,<NA>,<NA>,2018,10,1
1,I68793678,Software,<NA>,4 - Low,2018-10-03 03:11:35,2018-10-04 09:02:03,Closed,OfficeSuiteApplications-Lev2,Samuel Taylor,<NA>,<NA>,2018,10,1
2,I68793679,Software,<NA>,4 - Low,2018-10-03 03:05:13,2018-10-03 03:05:13,Closed,TechServiceDesk-APAC-NonEnglish-L1,Lily Parker,<NA>,<NA>,2018,10,1
3,I68793715,Software,<NA>,3 - Moderate,2018-10-03 04:29:41,2018-10-03 07:25:39,Closed,SFA-UserManagement-Lev2,Ethan Young,<NA>,<NA>,2018,10,1
4,I68793746,Software,<NA>,4 - Low,2018-10-03 05:37:06,2018-11-23 12:59:58,Closed,SFA-SQMC-Lev2,William Taylor,<NA>,<NA>,2018,10,1


##**7.Create marts**

In [15]:
# Avg resolution by category & priority
con.execute("""
CREATE OR REPLACE TABLE avg_resolution_category_priority AS
SELECT
  category,
  priority,
  COUNT(*) AS ticket_count,
  AVG(resolution_time_hrs) AS avg_resolution_time_hrs
FROM stg_tickets
WHERE resolution_time_hrs IS NOT NULL
GROUP BY 1,2
ORDER BY category, priority;
""")

# Closure rate by assigned group
con.execute("""
CREATE OR REPLACE TABLE closure_rate_assigned_group AS
SELECT
  assigned_group,
  COUNT(*) AS total_tickets,
  SUM(CASE WHEN lower(status) LIKE '%closed%' OR lower(status) LIKE '%resolved%' OR is_closed=1 THEN 1 ELSE 0 END) AS closed_tickets,
  ROUND(100.0 * SUM(CASE WHEN lower(status) LIKE '%closed%' OR lower(status) LIKE '%resolved%' OR is_closed=1 THEN 1 ELSE 0 END) / NULLIF(COUNT(*),0), 2) AS closure_rate_pct
FROM stg_tickets
GROUP BY 1
ORDER BY closure_rate_pct DESC;
""")

# Monthly ticket summary
con.execute("""
CREATE OR REPLACE TABLE monthly_ticket_summary AS
SELECT
  date_trunc('month', created_ts)::DATE AS month_start,
  COUNT(*) AS total_tickets,
  ROUND(AVG(resolution_time_hrs), 2) AS avg_resolution_time_hrs,
  ROUND(100.0 * SUM(CASE WHEN lower(status) LIKE '%closed%' OR lower(status) LIKE '%resolved%' OR is_closed=1 THEN 1 ELSE 0 END) / NULLIF(COUNT(*),0),2) AS closure_rate_pct
FROM stg_tickets
GROUP BY 1
ORDER BY 1;
""")

print("Marts created:")
print(con.execute("SHOW TABLES").df())


Marts created:
                               name
0  avg_resolution_category_priority
1       closure_rate_assigned_group
2            monthly_ticket_summary
3                       raw_tickets
4                       stg_tickets


##**8.Preview marts**

In [16]:
print("Monthly summary:")
display(con.execute("SELECT * FROM monthly_ticket_summary LIMIT 10").df())

print("Avg resolution:")
display(con.execute("SELECT * FROM avg_resolution_category_priority LIMIT 10").df())

print("Closure rates:")
display(con.execute("SELECT * FROM closure_rate_assigned_group LIMIT 10").df())


Monthly summary:


,month_start,total_tickets,avg_resolution_time_hrs,closure_rate_pct
0,2018-10-01,1870,NaN,97.38


Avg resolution:


,category,priority,ticket_count,avg_resolution_time_hrs


Closure rates:


,assigned_group,total_tickets,closed_tickets,closure_rate_pct
0,RPA-Lev2-L3,8,8.0,100.0
1,NPIPackPortfolio-CHAMPS-Lev2,14,14.0,100.0
2,Ariba-Expnce-Lev2,37,37.0,100.0
3,Deskside-Peringan-L1,4,4.0,100.0
4,First-SFA-Lev2,11,11.0,100.0
5,SFA-Workflow-Lev2,25,25.0,100.0
6,SFA-SFS-Finance-Lev2,98,98.0,100.0
7,Deskside-Athens-L1,2,2.0,100.0
8,UserAccount-Lev2,21,21.0,100.0
9,SFA-Security-Lev2,11,11.0,100.0


In [21]:
import duckdb, os

# Reconnect to the DuckDB database file
con = duckdb.connect(DB_PATH)

print("Reconnected to:", DB_PATH)
print("Available tables:", con.execute("SHOW TABLES").df())


Reconnected to: /content/drive/MyDrive/colab_ticket_pipeline/tickets.duckdb
Available tables:                                name
0  avg_resolution_category_priority
1       closure_rate_assigned_group
2            monthly_ticket_summary
3                       raw_tickets
4                       stg_tickets


##**9.Visualizations**

In [23]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# ===================== KPIs =====================
df_kpi = con.execute("""
    SELECT
      COUNT(*) AS total_tickets,
      ROUND(AVG(resolution_time_hrs),2) AS avg_resolution_time,
      ROUND(100.0 * SUM(CASE WHEN is_closed=1 OR lower(status) LIKE '%closed%' OR lower(status) LIKE '%resolved%' THEN 1 ELSE 0 END) / NULLIF(COUNT(*),0),2) AS closure_rate
    FROM stg_tickets
""").df()

total_tickets = int(df_kpi['total_tickets'][0])
avg_res_time = df_kpi['avg_resolution_time'][0]
closure_rate = df_kpi['closure_rate'][0]

# Display KPIs
fig_kpi = go.Figure()

fig_kpi.add_trace(go.Indicator(
    mode="number",
    value=total_tickets,
    title={"text": "Total Tickets"},
    domain={'x': [0, 0.3], 'y': [0, 1]}
))

fig_kpi.add_trace(go.Indicator(
    mode="number",
    value=avg_res_time if avg_res_time else 0,
    title={"text": "Avg Resolution (hrs)"},
    domain={'x': [0.35, 0.65], 'y': [0, 1]}
))

fig_kpi.add_trace(go.Indicator(
    mode="number",
    value=closure_rate,
    title={"text": "Closure Rate (%)"},
    domain={'x': [0.7, 1], 'y': [0, 1]}
))

fig_kpi.update_layout(title="📊 Key Ticket Metrics")
fig_kpi.show()

# ===================== Visualizations =====================

# 1. Monthly trend
df_month = con.execute("""
    SELECT month_start, total_tickets, closure_rate_pct
    FROM monthly_ticket_summary
    ORDER BY month_start
""").df()

if not df_month.empty:
    fig = go.Figure()
    fig.add_trace(go.Bar(x=df_month['month_start'], y=df_month['total_tickets'], name="Total Tickets"))
    fig.add_trace(go.Scatter(x=df_month['month_start'], y=df_month['closure_rate_pct'],
                             mode="lines+markers", name="Closure Rate %", yaxis="y2"))
    fig.update_layout(
        title="Monthly Tickets vs Closure Rate",
        yaxis=dict(title="Total Tickets"),
        yaxis2=dict(title="Closure Rate %", overlaying="y", side="right")
    )
    fig.show()

# 2. Heatmap: Avg resolution by category & priority
df_heat = con.execute("""
    SELECT category, priority, ROUND(avg_resolution_time_hrs,2) as avg_res
    FROM avg_resolution_category_priority
""").df()

if not df_heat.empty:
    fig = px.density_heatmap(df_heat, x="priority", y="category", z="avg_res",
                             color_continuous_scale="Reds",
                             title="Avg Resolution Time (hrs) by Category & Priority")
    fig.show()

# 3. Closure rate by group
df_closure = con.execute("""
    SELECT assigned_group, closure_rate_pct
    FROM closure_rate_assigned_group
    ORDER BY closure_rate_pct ASC
""").df()

if not df_closure.empty:
    fig = px.bar(df_closure, x="closure_rate_pct", y="assigned_group",
                 orientation="h", title="Closure Rate by Group",
                 labels={"closure_rate_pct":"Closure Rate %","assigned_group":"Group"})
    fig.show()

# 4. Resolution time distribution
df_res = con.execute("""
    SELECT resolution_time_hrs FROM stg_tickets WHERE resolution_time_hrs IS NOT NULL
""").df()

if not df_res.empty:
    fig = px.histogram(df_res, x="resolution_time_hrs", nbins=30,
                       title="Distribution of Resolution Times (hrs)",
                       labels={"resolution_time_hrs":"Resolution Time (hrs)"})
    fig.show()


In [24]:
from google.colab import files
con.close()
if os.path.exists(DB_PATH):
    files.download(DB_PATH)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>